## Make coco splits

In [1]:
from pylabel import importer, exporter, dataset
import numpy as np
from sklearn.model_selection import train_test_split
import json
import pandas as pd

#### Save the entire dataset to coco

In [2]:
voc_path = '/home/gridsan/manderson/ovdsat/data/dior'

In [3]:
mydataset = importer.ImportVOC(path = voc_path+'/Annotations', path_to_images=voc_path+'/JPEGImages', name='DIOR')

Importing VOC files...: 100%|██████████| 17403/17403 [01:29<00:00, 194.63it/s]


In [6]:
mydataset.df.head()

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_iscrowd,ann_keypoints,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,17635.jpg,NaN,0,800,800,3,0,650.0,286.0,...,NaN,NaN,Unspecified,,,0,groundtrackfield,NaN,,1
1,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,17635.jpg,NaN,0,800,800,3,0,571.0,440.0,...,NaN,NaN,Unspecified,,,1,baseballfield,NaN,,1
2,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,17635.jpg,NaN,0,800,800,3,0,681.0,101.0,...,NaN,NaN,Unspecified,,,2,bridge,NaN,,1
3,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,12474.jpg,NaN,1,800,800,3,0,202.0,237.0,...,NaN,NaN,Unspecified,,,3,Expressway-toll-station,NaN,,1
4,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,12474.jpg,NaN,1,800,800,3,0,430.0,697.0,...,NaN,NaN,Unspecified,,,4,vehicle,NaN,,1


In [8]:
exp = exporter.Export(mydataset)
exp.ExportToCoco(output_path=f'{voc_path}/dior_coco_all.json', cat_id_index=0)

/home/gridsan/manderson/.conda/envs/test/lib/python3.9/site-packages/pylabel/exporter.py:767: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r"^\s*$", np.nan, regex=True)
Exporting to COCO file...: 100%|██████████| 38320/38320 [00:31<00:00, 1200.08it/s]


['/home/gridsan/manderson/ovdsat/data/dior/dior_coco_all.json']

### Generate splits for N=5,10,30, finetune, and validation

In [3]:
# Load your COCO dataset
coco_json = "/home/gridsan/manderson/ovdsat/data/dior/dior_coco_all.json"
path_to_images = "/home/gridsan/manderson/ovdsat/data/dior/JPEGImages"   # folder with your images

all_dataset = importer.ImportCoco(
    path=coco_json,
    path_to_images=path_to_images,
    name="dior"
)

# Get the dataframe
df = all_dataset.df
print(len(df))
df.head()

38320


,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_iscrowd,ann_keypoints,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,17635.jpg,,0,800,800,3,0,650.0,286.0,...,0.0,,Unspecified,,,0,groundtrackfield,,,1
1,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,17635.jpg,,0,800,800,3,0,571.0,440.0,...,0.0,,Unspecified,,,1,baseballfield,,,1
2,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,17635.jpg,,0,800,800,3,0,681.0,101.0,...,0.0,,Unspecified,,,2,bridge,,,1
3,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,12474.jpg,,1,800,800,3,0,202.0,237.0,...,0.0,,Unspecified,,,3,Expressway-toll-station,,,1
4,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,12474.jpg,,1,800,800,3,0,430.0,697.0,...,0.0,,Unspecified,,,4,vehicle,,,1


In [4]:
save_dir = '/home/gridsan/manderson/ovdsat/data/dior'

for M in [1, 2, 3, 4, 5]:
    img_ids = set()
    prev_samples = {}  # track selected samples per category
    
    for N in [1, 3, 5, 10, 30]:
        train_file = f'{save_dir}/train_coco_subset_N{N}-{M}.json'
        val_file = f'{save_dir}/val_coco-{M}.json'
        finetune_val_file = f'{save_dir}/train_coco_finetune_val-{M}.json'
        
        N_samples_list = []

        for value, group in df.groupby('cat_id'):
            # start with previous samples for this category, if any
            prev_df = prev_samples.get(value, pd.DataFrame(columns=group.columns))

            # number of new samples to add for this N
            needed = max(0, N - len(prev_df))
            if needed > 0:
                new_samples = group.loc[~group['img_id'].isin(prev_df['img_id'])].sample(
                    n=min(needed, len(group) - len(prev_df)), replace=False
                )
                current_df = pd.concat([prev_df, new_samples], ignore_index=True)
            else:
                current_df = prev_df

            # save for next round
            prev_samples[value] = current_df
            N_samples_list.append(current_df)
            img_ids.update(current_df['img_id'].tolist())

        print(f"M={M}, N={N}, total unique imgs={len(img_ids)}")

        # combine and export
        N_samples_df = pd.concat(N_samples_list, ignore_index=True).reset_index(drop=True)
        pylabel_df_N = dataset.Dataset(N_samples_df)
        exp = exporter.Export(pylabel_df_N)
        exp.ExportToCoco(output_path=train_file, cat_id_index=0)

        with open(train_file, 'r') as f:
            train_data = json.load(f)
        train_data['categories'] = sorted(train_data['categories'], key=lambda x: x['id'])
        with open(f'{train_file[:-5]}.json', 'w') as f:
            json.dump(train_data, f, indent=4)
        print(f'Saved {train_file[:-5]}.json')

    # Remaining images (not in any N-shot)
    remaining_df = df[~df['img_id'].isin(img_ids)]
    remaining_img_ids = remaining_df['img_id'].unique()

    # Split into validation and finetune sets
    ft_percent = 0.1
    val_ids, ft_ids = train_test_split(remaining_img_ids, test_size=ft_percent)

    # --- Validation set ---
    val_df = df[df['img_id'].isin(val_ids)].reset_index(drop=True)
    exp = exporter.Export(dataset.Dataset(val_df))
    exp.ExportToCoco(output_path=val_file, cat_id_index=0)

    # --- Finetune validation set ---
    ft_df = df[df['img_id'].isin(ft_ids)].reset_index(drop=True)
    exp = exporter.Export(dataset.Dataset(ft_df))
    exp.ExportToCoco(output_path=finetune_val_file, cat_id_index=0)

    # Sort categories in both
    for fpath in [finetune_val_file, val_file]:
        with open(fpath, 'r') as f:
            data = json.load(f)
        data['categories'] = sorted(data['categories'], key=lambda x: x['id'])
        with open(f'{fpath[:-5]}.json', 'w') as f:
            json.dump(data, f, indent=4)
        print(f'Saved {fpath[:-5]}.json')


/state/partition1/slurm_tmp/3515210.0.0/ipykernel_3003597/3374553936.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_df = pd.concat([prev_df, new_samples], ignore_index=True)
/state/partition1/slurm_tmp/3515210.0.0/ipykernel_3003597/3374553936.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_df = pd.concat([prev_df, new_samples], ignore_index=True)
/state/partition1/slurm_tmp/3515210.0.0/ipykernel_3003597/3374553936.py:24: FutureWarning: The behavior of DataFrame concate

M=1, N=1, total unique imgs=20


Exporting to COCO file...: 100%|██████████| 20/20 [00:00<00:00, 401.74it/s]

Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N1-1.json



/home/gridsan/manderson/.conda/envs/test/lib/python3.9/site-packages/pylabel/exporter.py:767: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r"^\s*$", np.nan, regex=True)


...

M=5, N=30, total unique imgs=591


Exporting to COCO file...: 100%|██████████| 600/600 [00:00<00:00, 2422.02it/s]


Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N30-5.json


Exporting to COCO file...: 100%|██████████| 3626/3626 [00:01<00:00, 2502.49it/s]


Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_finetune_val-5.json
Saved /home/gridsan/manderson/ovdsat/data/dior/val_coco-5.json
